In [1]:
import sys
import os
import fastText

from validation import compute_f1

from keras.models import load_model
from keras.callbacks import ModelCheckpoint, Callback

import models
import utils

/home/gwiedemann/miniconda3/envs/kerasenv/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=1

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=1


In [3]:
trainSentences = utils.get_sentences_germeval('../data/GermEVAL/NER-de-train.tsv')
devSentences = utils.get_sentences_germeval('../data/GermEVAL/NER-de-dev.tsv')
testSentences = utils.get_sentences_germeval('../data/GermEVAL/NER-de-test.tsv')

# trainSentences = get_sentences('../data/CONLL/deu/deu_utf.train')
# devSentences = get_sentences('../data/CONLL/deu/deu_utf.testa')
# testSentences = get_sentences('../data/CONLL/deu/deu_utf.testb')

print(len(trainSentences))
print(len(devSentences))
print(len(testSentences))


24000
2200
5100


In [4]:
print(testSentences[0])

[['1951', 'O'], ['bis', 'O'], ['1953', 'O'], ['wurde', 'O'], ['der', 'O'], ['nördliche', 'O'], ['Teil', 'O'], ['als', 'O'], ['Jugendburg', 'O'], ['des', 'O'], ['Kolpingwerkes', 'B-OTH'], ['gebaut', 'O'], ['.', 'O']]


In [5]:
labelSet = set()
characters= set()
models.max_sequence_length = 0

for dataset in [trainSentences, devSentences, testSentences]:
    for sentence in dataset:
        for word, label in sentence:
            for char in word:
                characters.add(char)
            labelSet.add(label)
        if len(sentence) > models.max_sequence_length:
            models.max_sequence_length = len(sentence)

In [6]:
print(len(labelSet))
print(models.max_sequence_length)

25
56


In [7]:
# :: Create a mapping for the labels ::
models.label2Idx = {"PADDING_TOKEN":0}
for label in labelSet:
    models.label2Idx[label] = len(models.label2Idx)

In [8]:
print(models.label2Idx)

{'I-OTH': 1, 'B-PERderiv': 2, 'B-OTH': 16, 'B-ORGpart': 3, 'I-OTHderiv': 17, 'B-LOC': 5, 'B-PERpart': 6, 'B-PER': 24, 'B-LOCpart': 7, 'I-PER': 18, 'I-PERpart': 4, 'I-ORG': 19, 'B-OTHpart': 20, 'B-ORG': 8, 'B-ORGderiv': 23, 'I-ORGderiv': 10, 'I-ORGpart': 21, 'I-PERderiv': 9, 'O': 11, 'I-LOCpart': 12, 'I-OTHpart': 13, 'B-LOCderiv': 14, 'PADDING_TOKEN': 0, 'I-LOC': 15, 'I-LOCderiv': 25, 'B-OTHderiv': 22}


In [9]:
# :: Hard coded case lookup ::
models.case2Idx = {'PADDING_TOKEN':0, 'numeric': 1, 'allLower':2, 'allUpper':3, 'initialUpper':4, 'other':5, 'mainly_numeric':6, 'contains_digit': 7}

In [10]:
print(models.case2Idx)

{'mainly_numeric': 6, 'contains_digit': 7, 'other': 5, 'PADDING_TOKEN': 0, 'allLower': 2, 'numeric': 1, 'allUpper': 3, 'initialUpper': 4}


In [11]:
print(type(trainSentences))

<class 'list'>


In [12]:
print(trainSentences[0])

[['Schartau', 'B-PER'], ['sagte', 'O'], ['dem', 'O'], ['"', 'O'], ['Tagesspiegel', 'B-ORG'], ['"', 'O'], ['vom', 'O'], ['Freitag', 'O'], [',', 'O'], ['Fischer', 'B-PER'], ['sei', 'O'], ['"', 'O'], ['in', 'O'], ['einer', 'O'], ['Weise', 'O'], ['aufgetreten', 'O'], [',', 'O'], ['die', 'O'], ['alles', 'O'], ['andere', 'O'], ['als', 'O'], ['überzeugend', 'O'], ['war', 'O'], ['"', 'O'], ['.', 'O']]


In [13]:
models.char2Idx={"PADDING_TOKEN":0, "<S>":1, "</S>":2, "<W>":3, "</W>":4}
for char in characters:
    models.char2Idx[char] = len(models.char2Idx)
models.char2Idx['UNKNOWN'] = len(models.char2Idx)
print(models.char2Idx)

{'í': 5, 'Å': 6, '-': 7, '²': 8, '術': 63, 'ν': 181, 'ю': 197, ':': 12, 'α': 117, 'г': 10, '造': 14, 'У': 15, '©': 16, 'Ġ': 320, 'κ': 17, 'ş': 20, 'ʻ': 311, '\xad': 21, 'т': 22, '€': 283, 'É': 24, '(': 25, 'オ': 26, 'M': 27, 'τ': 28, '[': 29, 'x': 30, 'И': 67, '樓': 263, 'Ş': 173, 'ŏ': 36, 'ř': 33, 'g': 34, '\x92': 35, '寝': 290, 'Π': 37, '算': 244, '\x96': 38, '_': 174, 'ħ': 39, 'з': 40, '懿': 41, '\x9a': 42, 'έ': 45, '.': 46, 'ą': 70, ']': 47, '\x80': 48, 'Ü': 49, 'С': 50, 'ú': 51, 'н': 128, 'ǒ': 52, 'ø': 54, '殿': 55, 'γ': 56, 'ب': 59, 'ü': 60, 'л': 279, 'к': 13, 'Á': 61, 'é': 62, 'B': 65, 'ο': 178, 'ῦ': 66, 'I': 68, 'š': 69, '>': 71, '“': 287, '9': 72, 'ż': 73, 'ž': 74, 'Λ': 211, '7': 76, 'Y': 77, 'X': 80, '%': 79, 'Þ': 216, '¹': 18, 'ť': 89, '„': 82, '守': 84, 'b': 85, 'w': 140, 'ї': 87, 'V': 88, 'Ž': 242, 'h': 90, 'Î': 91, '대': 292, '#': 92, 'i': 19, 'β': 93, '‘': 94, 'İ': 318, '章': 96, 's': 97, '@': 101, 'σ': 103, 'z': 100, '…': 102, 'ـ': 104, 'ế': 23, '$': 105, 'е': 107, 'U': 78, 'æ': 1

In [14]:
print(trainSentences[0]) 

[['Schartau', 'B-PER'], ['sagte', 'O'], ['dem', 'O'], ['"', 'O'], ['Tagesspiegel', 'B-ORG'], ['"', 'O'], ['vom', 'O'], ['Freitag', 'O'], [',', 'O'], ['Fischer', 'B-PER'], ['sei', 'O'], ['"', 'O'], ['in', 'O'], ['einer', 'O'], ['Weise', 'O'], ['aufgetreten', 'O'], [',', 'O'], ['die', 'O'], ['alles', 'O'], ['andere', 'O'], ['als', 'O'], ['überzeugend', 'O'], ['war', 'O'], ['"', 'O'], ['.', 'O']]


In [15]:
models.ft = fastText.load_model("../embeddings/wiki.de.bin")

In [16]:
print(models.nb_embedding_dims)
print(len(trainSentences[0]))

300
25


In [17]:
models.idx2Label = {v: k for k, v in models.label2Idx.items()}
print(len(models.label2Idx))
print(len(models.idx2Label))

26
26


In [18]:
import numpy as np
from sklearn.decomposition import PCA

char_embeddings_full = []
models.idx2Char = {v: k for k, v in models.char2Idx.items()}
for idx in models.idx2Char:
    # print(idx)
    char_embeddings_full.append(models.ft.get_word_vector(models.idx2Char[idx]))

char_embeddings_ft_dimensional = np.transpose(np.asarray(char_embeddings_full))

print(models.nb_char_embeddings)
print(char_embeddings_ft_dimensional.shape)

from sklearn.preprocessing import StandardScaler
char_embeddings_ft_dimensional = StandardScaler().fit_transform(char_embeddings_ft_dimensional)

# reduce dimensions
pca_embeddings = {}
pca =  PCA(n_components = models.nb_char_embeddings)
X_fit = pca.fit_transform(char_embeddings_ft_dimensional)
U1 = pca.components_

52
(300, 334)


In [19]:
print(U1.shape)
ft_char_embeddings = np.transpose(U1)
print(ft_char_embeddings.shape)

print(ft_char_embeddings[models.char2Idx['?']])
print(ft_char_embeddings[models.char2Idx['!']])
models.ft_char_embeddings = ft_char_embeddings

(52, 334)
(334, 52)
[ 0.06097103 -0.08364963  0.0420165  -0.05594541 -0.12055056  0.0753653
  0.04046584 -0.00258806 -0.03529835 -0.00083467 -0.00248946 -0.01931428
  0.00973782 -0.00163436  0.00931013  0.03026976  0.01325981  0.04547809
  0.01414036  0.01067183  0.00755442  0.00483857 -0.00972926  0.0324423
 -0.02299908  0.0499439   0.0489522   0.03431637  0.11215851  0.06449113
  0.02677515  0.01958173  0.02194596  0.05583817  0.01828234 -0.00787334
  0.06406736 -0.02727523 -0.04293418 -0.03721522 -0.01577569  0.0299582
  0.01945353 -0.07267907  0.03878674 -0.07088879 -0.02779063 -0.04633658
  0.09528352  0.04456796 -0.00797106 -0.0847523 ]
[ 0.06002061 -0.08579992  0.03255959 -0.03155637 -0.11854998  0.03731262
  0.09541047 -0.00375484 -0.01461903  0.02828638  0.03400048 -0.00081043
 -0.0027184  -0.01664059  0.02811808  0.00161681  0.04817512  0.01968311
  0.04087862  0.00571308  0.0027742  -0.05689412 -0.03675508  0.04068714
 -0.02999051 -0.02272069  0.01600836 -0.00285114  0.07411

# Test Model

In [20]:
#import importlib
#importlib.reload(models)

In [21]:
tmp_model_filename = 'tmp_3cnn_bi-lstm_pretrained.h5'
# checkpoint = ModelCheckpoint(tmp_model_filename, verbose=1, save_best_only = True, monitor = 'val_acc')
history = utils.F1History(tmp_model_filename, devSet = devSentences)
model = models.get_model_3cnn()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
char_input (InputLayer)         (None, None, 52)     0                                            
__________________________________________________________________________________________________
char_embedding (TimeDistributed (None, None, 52, 32) 10688       char_input[0][0]                 
__________________________________________________________________________________________________
time_distributed_1 (TimeDistrib (None, None, 52, 32) 3104        char_embedding[0][0]             
__________________________________________________________________________________________________
time_distributed_4 (TimeDistrib (None, None, 52, 32) 4128        char_embedding[0][0]             
__________________________________________________________________________________________________
time_distr

In [22]:
# import importlib
# importlib.reload(utils)
#cprint(models.max_sequence_length)

In [23]:
model.fit_generator(
    utils.NerSequence(trainSentences, shuffle_data=True, batch_size=32), 
    validation_data = utils.NerSequence(devSentences, batch_size=256), 
    epochs = 10, callbacks = [history]
)

Epoch 1/10
749/750 [============================>.] - ETA: 0s - loss: 0.0653 - acc: 0.9818
New maximum F1 score: 0.7508005274062913 (before: 0) Saving to tmp_3cnn_bi-lstm_pretrained.h5
750/750 [==============================] - 232s 309ms/step - loss: 0.0653 - acc: 0.9818 - val_loss: 0.0248 - val_acc: 0.9887
Epoch 2/10
749/750 [============================>.] - ETA: 0s - loss: 0.0159 - acc: 0.9883
New maximum F1 score: 0.7821800947867298 (before: 0.7508005274062913) Saving to tmp_3cnn_bi-lstm_pretrained.h5
750/750 [==============================] - 221s 294ms/step - loss: 0.0159 - acc: 0.9883 - val_loss: 0.0010 - val_acc: 0.9899
Epoch 3/10
749/750 [============================>.] - ETA: 0s - loss: -0.0097 - acc: 0.9898
New maximum F1 score: 0.802118804388952 (before: 0.7821800947867298) Saving to tmp_3cnn_bi-lstm_pretrained.h5
750/750 [==============================] - 221s 294ms/step - loss: -0.0097 - acc: 0.9898 - val_loss: -0.0240 - val_acc: 0.9906
Epoch 4/10
750/750 [==============

In [24]:
print(history.acc)
print(history.f1_scores)

[0.9887094150890003, 0.9898701184446161, 0.9905925315076655, 0.9894480486349626, 0.9908847394856539, 0.9914529241215099, 0.9908685046976263, 0.9911850632320751, 0.9908116804469715, 0.9915340978449041]
[0.7508005274062913, 0.7821800947867298, 0.802118804388952, 0.7946947674418605, 0.8114496768236381, 0.8211351755041075, 0.8159259259259259, 0.8185013876040703, 0.8167924875713497, 0.8262002232973576]


In [25]:
model.load_weights(tmp_model_filename)

In [26]:
model.fit_generator(
    utils.NerSequence(trainSentences, shuffle_data=True, batch_size=2048), 
    validation_data = utils.NerSequence(devSentences, batch_size=256), 
    epochs = 10, callbacks = [history]
)

Epoch 1/10
11/12 [==========================>...] - ETA: 5s - loss: -0.2172 - acc: 0.9945 
New maximum F1 score: 0.8235511258767074 (before: 0) Saving to tmp_3cnn_bi-lstm_pretrained.h5
12/12 [==============================] - 91s 8s/step - loss: -0.2172 - acc: 0.9945 - val_loss: -0.2088 - val_acc: 0.9914
Epoch 2/10
12/12 [==============================] - 70s 6s/step - loss: -0.2181 - acc: 0.9947 - val_loss: -0.2094 - val_acc: 0.9913
Epoch 3/10
12/12 [==============================] - 69s 6s/step - loss: -0.2187 - acc: 0.9948 - val_loss: -0.2098 - val_acc: 0.9914
Epoch 4/10
11/12 [==========================>...] - ETA: 3s - loss: -0.2191 - acc: 0.9948
New maximum F1 score: 0.825654257279764 (before: 0.8235511258767074) Saving to tmp_3cnn_bi-lstm_pretrained.h5
12/12 [==============================] - 67s 6s/step - loss: -0.2192 - acc: 0.9948 - val_loss: -0.2102 - val_acc: 0.9915
Epoch 5/10
11/12 [==========================>...] - ETA: 3s - loss: -0.2198 - acc: 0.9949
New maximum F1 scor

In [27]:
true_labels, pred_labels = utils.predict_sequences(model, testSentences)
print(compute_f1(pred_labels, true_labels, models.idx2Label))

(0.8182111200644642, 0.8217869860796374, 0.8199951546475006)


# Experiments

In [ ]:
f = open('results_3cnn_nodense.txt', 'a')
for run_i in range(10):
    print("Run " + str(run_i))
    
    tmp_model_filename = 'tmp_generator_NER_3cnn_nodense_best.' + str(run_i) + '.h5'

    history = utils.F1History(tmp_model_filename, devSet=devSentences)

    model = models.get_model_3cnn()
    model.fit_generator(
        utils.NerSequence(trainSentences, shuffle_data=True, batch_size=32), 
        validation_data = utils.NerSequence(devSentences, batch_size=512), 
        epochs = 10, callbacks = [history]
    )
    
    model.load_weights(tmp_model_filename)
    
    model.fit_generator(
        utils.NerSequence(trainSentences, shuffle_data=True, batch_size=2048), 
        validation_data = utils.NerSequence(devSentences, batch_size=512), 
        epochs = 15, callbacks = [history]
    )
    
    model.load_weights(tmp_model_filename)
    
    true_labels, pred_labels = utils.predict_sequences(model, testSentences)
    
    pre, rec, f1 = compute_f1(pred_labels, true_labels, models.idx2Label)
    f.write(str(run_i) + "\t" + str(pre) + "\t" + str(rec) +  "\t" + str(f1))
    f.write("\n")
    f.flush()
f.close()

Run 0
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
char_input (InputLayer)         (None, None, 52)     0                                            
__________________________________________________________________________________________________
char_embedding (TimeDistributed (None, None, 52, 32) 10688       char_input[0][0]                 
__________________________________________________________________________________________________
time_distributed_28 (TimeDistri (None, None, 52, 32) 3104        char_embedding[0][0]             
__________________________________________________________________________________________________
time_distributed_31 (TimeDistri (None, None, 52, 32) 4128        char_embedding[0][0]             
__________________________________________________________________________________________________
time

12/12 [==============================] - 79s 7s/step - loss: -0.1946 - acc: 0.9946 - val_loss: -0.1869 - val_acc: 0.9916
Epoch 5/15
12/12 [==============================] - 77s 6s/step - loss: -0.1952 - acc: 0.9946 - val_loss: -0.1873 - val_acc: 0.9916
Epoch 6/15
11/12 [==========================>...] - ETA: 4s - loss: -0.1956 - acc: 0.9947
New maximum F1 score: 0.829520295202952 (before: 0.8290519312511551) Saving to tmp_generator_NER_3cnn_nodense_best.0.h5
12/12 [==============================] - 76s 6s/step - loss: -0.1956 - acc: 0.9947 - val_loss: -0.1878 - val_acc: 0.9916
Epoch 7/15
11/12 [==========================>...] - ETA: 4s - loss: -0.1963 - acc: 0.9947
New maximum F1 score: 0.829673371470751 (before: 0.829520295202952) Saving to tmp_generator_NER_3cnn_nodense_best.0.h5
12/12 [==============================] - 81s 7s/step - loss: -0.1963 - acc: 0.9948 - val_loss: -0.1883 - val_acc: 0.9916
Epoch 8/15
12/12 [==============================] - 83s 7s/step - loss: -0.1969 - acc:

Epoch 5/10
749/750 [============================>.] - ETA: 0s - loss: -0.0684 - acc: 0.9914
New maximum F1 score: 0.816057293629853 (before: 0.8036546708931569) Saving to tmp_generator_NER_3cnn_nodense_best.1.h5
750/750 [==============================] - 258s 344ms/step - loss: -0.0684 - acc: 0.9914 - val_loss: -0.0802 - val_acc: 0.9909
Epoch 6/10
749/750 [============================>.] - ETA: 0s - loss: -0.0960 - acc: 0.9919
New maximum F1 score: 0.819757688723206 (before: 0.816057293629853) Saving to tmp_generator_NER_3cnn_nodense_best.1.h5
750/750 [==============================] - 263s 350ms/step - loss: -0.0960 - acc: 0.9919 - val_loss: -0.1074 - val_acc: 0.9914
Epoch 7/10
749/750 [============================>.] - ETA: 0s - loss: -0.1238 - acc: 0.9924
New maximum F1 score: 0.8220402084884586 (before: 0.819757688723206) Saving to tmp_generator_NER_3cnn_nodense_best.1.h5
750/750 [==============================] - 260s 347ms/step - loss: -0.1238 - acc: 0.9924 - val_loss: -0.1340 - 

Epoch 1/10
749/750 [============================>.] - ETA: 0s - loss: 0.0685 - acc: 0.9820
New maximum F1 score: 0.7461318051575931 (before: 0) Saving to tmp_generator_NER_3cnn_nodense_best.2.h5
750/750 [==============================] - 250s 333ms/step - loss: 0.0684 - acc: 0.9820 - val_loss: 0.0293 - val_acc: 0.9884
Epoch 2/10
749/750 [============================>.] - ETA: 0s - loss: 0.0193 - acc: 0.9883
New maximum F1 score: 0.7803379416282642 (before: 0.7461318051575931) Saving to tmp_generator_NER_3cnn_nodense_best.2.h5
750/750 [==============================] - 227s 303ms/step - loss: 0.0193 - acc: 0.9883 - val_loss: 0.0035 - val_acc: 0.9897
Epoch 3/10
749/750 [============================>.] - ETA: 0s - loss: -0.0072 - acc: 0.9897
New maximum F1 score: 0.7954587753582729 (before: 0.7803379416282642) Saving to tmp_generator_NER_3cnn_nodense_best.2.h5
750/750 [==============================] - 230s 307ms/step - loss: -0.0072 - acc: 0.9898 - val_loss: -0.0210 - val_acc: 0.9904
Epo

Epoch 1/10
749/750 [============================>.] - ETA: 0s - loss: 0.0647 - acc: 0.9821
New maximum F1 score: 0.7454933126574917 (before: 0) Saving to tmp_generator_NER_3cnn_nodense_best.3.h5
750/750 [==============================] - 249s 332ms/step - loss: 0.0646 - acc: 0.9821 - val_loss: 0.0245 - val_acc: 0.9884
Epoch 2/10
749/750 [============================>.] - ETA: 0s - loss: 0.0144 - acc: 0.9883
New maximum F1 score: 0.77794852093738 (before: 0.7454933126574917) Saving to tmp_generator_NER_3cnn_nodense_best.3.h5
750/750 [==============================] - 247s 329ms/step - loss: 0.0144 - acc: 0.9883 - val_loss: -5.4810e-04 - val_acc: 0.9895
Epoch 3/10
749/750 [============================>.] - ETA: 0s - loss: -0.0119 - acc: 0.9898
New maximum F1 score: 0.7941176470588234 (before: 0.77794852093738) Saving to tmp_generator_NER_3cnn_nodense_best.3.h5
750/750 [==============================] - 264s 353ms/step - loss: -0.0119 - acc: 0.9898 - val_loss: -0.0251 - val_acc: 0.9903
Ep

# Save final model

In [ ]:
import shutil, json
# copy file for best run
shutil.copyfile('tmp_3cnn_bi-lstm.h5', '../models/final_model_germeval_outer.h5')
with open("../models/final_model_germeval.indexes", "w") as f:
    json.dump([models.idx2Label, models.label2Idx, models.char2Idx, models.case2Idx], f)

In [ ]:
from keras_contrib.layers import CRF
def create_custom_objects():
    instanceHolder = {"instance": None}
    class ClassWrapper(CRF):
        def __init__(self, *args, **kwargs):
            instanceHolder["instance"] = self
            super(ClassWrapper, self).__init__(*args, **kwargs)
    def loss(*args):
        method = getattr(instanceHolder["instance"], "loss_function")
        return method(*args)
    def accuracy(*args):
        method = getattr(instanceHolder["instance"], "accuracy")
        return method(*args)
    return {"ClassWrapper": ClassWrapper ,"CRF": ClassWrapper, "loss": loss, "accuracy":accuracy}

finalmodel = load_model('../models/final_model_germeval_outer.h5', custom_objects=create_custom_objects())
true_labels, pred_labels = utils.predict_sequences(finalmodel, testSentences)
print(compute_f1(pred_labels, true_labels, models.idx2Label))

In [ ]:
f = open('germeval_output.tsv', 'w', encoding='UTF-8')
for i_sent, sent in enumerate(testSentences):
    for i_tok, tok in enumerate(sent):
        if tok[0] == 'PADDING_TOKEN':
            break
        correctlabel = models.idx2Label[true_labels[i_sent][i_tok]]
        guessedlabel = models.idx2Label[pred_labels[i_sent][i_tok]]
        line = "\t".join([str(i_tok+1), tok[0], correctlabel, correctlabel, guessedlabel, guessedlabel])
        f.write(line + '\n')
    f.write('\n')
f.close